## ML models

This notebook trains the following ML models:

1. Logistic Regressor
2. Decision Tree
3. Support-Vector Machine
4. K-Nearest Neighbours
5. Random Forests

as well as two boosting methods:

1. Extreme Gradient Boosting Machine
2. Light Gradient Boosting Machine

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor, VotingRegressor)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor as lgb
from xgboost import XGBRegressor as xgb

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (cross_validate, KFold, cross_val_score, train_test_split)
import optuna

In [2]:
kfold = KFold(n_splits=5)

In [3]:
def get_data(name:str='', SHUFFLE_FLAG:bool=False, NORM_FLAG:bool=True, random_state:int=42):
    '''
    Function to select data

    Arguments
    ---------
    name: str, (required)
        name of dataset to be returned
    SHUFFLE_FLAG: bool, (optional)
        Flag for if the data should be shuffled
    NORM_FLAG: bool, (optional)
        If the data should be normalized
    random_state: int, (optional)
        random_state
    
    Returns
    -------
    X: numpy.ndarray 
        training set 
    y: numpy.ndarray 
        test set
    '''
    
    if name is None:
        raise ValueError("Required argument 'name' is missing.")
    
    if name == "gaia":
        dir = '../data/Gaia DR3/gaia_lm_m_stars.parquet'
        data = pd.read_parquet(dir)
        if SHUFFLE_FLAG:
            df = shuffle(data)
        else:
            df = data
        X = np.vstack(df['flux'])
        y = np.vstack(df['Cat'])
        
        y = np.where(y == 'M', 1, y)
        y = np.where(y == 'LM', 0, y)

        y = y.astype(int)

        if NORM_FLAG:
            norm = np.linalg.norm(X,keepdims=True)
            X = X/norm
            

    elif name == 'apogee':
        dir = '../data/APOGEE'
        train_dir = dir + '/training_data.h5'
        tets_dir = dir +'/test_data.h5'

        with h5py.File(train_dir, 'r') as f:
            X = f['spectrum'][:]
            y = np.hstack((f['TEFF'],
                        f['LOGG'],
                        f['FE_H']))
        
        #TODO: add shuffle

        if NORM_FLAG:
            norm_dir = dir + '/mean_and_std.npy'
            norm_data = np.load(norm_dir)
            
            mean = norm_data[0]
            std = norm_data[1]
            y = (y-mean)/std

    return X, y

In [4]:
#X, y = get_data('gaia', SHUFFLE_FLAG=True)
X, y = get_data('apogee')

num_samples = X.shape[0]
spectrum_width = X.shape[1]

num_samples_m = np.count_nonzero(y)
num_samples_lm = len(y) - num_samples_m
num_classes = len(np.unique(y))

print("Total number of spectra:", num_samples)
print("Number of bins in each spectra:", spectrum_width)

Total number of spectra: 44784
Number of bins in each spectra: 7214


In [5]:
# naive splitting methods

split = 0.8

train_size = int(split * num_samples)

x_train, x_test = np.split(X, [train_size])
y_train, y_test = np.split(y, [train_size])

#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [6]:
#print("The dataset is divided into", len(x_train), "training samples and", len(x_test),"testing samples.")

## Decision Trees

In [7]:
mse_scores = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    
    x_train, x_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = DecisionTreeRegressor()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for fold {fold}: {mse}")
    mse_scores.append(mse)

print(np.mean(mse_scores))

## Random Forest

In [7]:
mse_scores = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    
    x_train, x_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = RandomForestRegressor()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for fold {fold}: {mse}")
    mse_scores.append(mse)

print(np.mean(mse_scores))

In [ ]:
def objective(trial):

    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 1 , 50)
    min_samples_split = trial.suggest_int('min_samples_split', 1, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)

    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

    rkf = RepeatedStratifiedKFold(n_splits = 5)
    score = cross_val_score(model, x_train, y_train.squeeze(1), cv=rkf, scoring='accuracy')

    return score


study = optuna.create_study(direction='maximize', study_name='xgb_model_training')
study.optimize(objective, n_trials=100)

## K-Nearest Neighbours

In [6]:
mse_scores = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    
    x_train, x_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = KNeighborsRegressor()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for fold {fold}: {mse}")
    mse_scores.append(mse)

print(np.mean(mse_scores))

Mean Squared Error for fold 0: 0.057355403900146484
Mean Squared Error for fold 1: 0.05811426043510437
Mean Squared Error for fold 2: 0.061873167753219604
Mean Squared Error for fold 3: 0.06017027795314789
Mean Squared Error for fold 4: 0.05607803165912628
0.058718227


## Light Gradient Boosting Machine

In [5]:
#-------------------initial naive implementation, needs a lot more tuning-------------------------------
mse_scores = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    
    x_train, x_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    lgb_model = lgb()
    model = MultiOutputRegressor(lgb_model)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for fold {fold}: {mse}")
    mse_scores.append(mse)

print(np.mean(mse_scores))

In [ ]:
def objective(trial):
    return

## Extreme Gradient Boosting Machine

In [5]:
#-------------------initial naive implementation, needs a lot more tuning-------------------------------

mse_scores = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    
    x_train, x_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    model = xgb()
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for fold {fold}: {mse}")
    mse_scores.append(mse)

print(np.mean(mse_scores))

Mean Squared Error for fold 0: 0.006877315696328878


KeyboardInterrupt: 

In [ ]:
def objective(trial):
    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree','gblinear']),
        'device': 'cuda',
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise','lossguide']),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'gamma' : trial.suggest_float('gamma', 1e-5, 0.5, log=True),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
    }

    params['n_estimators'] = 3000
    params['early_stopping_rounds'] = 50
    params['booster'] = 'gbtree'
    params["verbosity"] = 0
    params['tree_method'] = "hist"
    
    auc_scores = []

    for fold, (train_idx, valid_idx) in enumerate(kfold.split(X, y)):

        X_train_fold, X_valid_fold = pd.DataFrame(X).iloc[train_idx], pd.DataFrame(X).iloc[valid_idx]
        y_train_fold, y_valid_fold = pd.Series(y.squeeze(1)).iloc[train_idx], pd.Series(y.squeeze(1)).iloc[valid_idx]
                
        # Create and fit the model
        model = xgb(**params)
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_valid_fold, y_valid_fold)],verbose=False)

        # Predict class probabilities
        y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error for fold {fold}: {mse}")
    mse_scores.append(mse)

    return (np.mean(mse_scores))

study = optuna.create_study(direction='maximize', study_name='xgb_model_training')
study.optimize(objective, n_trials=200)